## **Ejercicios propuestos:**

* **Preprocesamiento:**
    1. Modificar n_steps
* **Arquitectura:**
    1. Modificar el número de neuronas
    2. Cambiar la función de activación de cada capa (relu, softmax, linear, tanh)
* **Entrenamiento:** 
    1. Modificar la cantidad de epochs
    2. Modificar el batch_size

# Paquetes

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib
matplotlib.use('Agg') 

from sklearn.model_selection import train_test_split

import tensorflow as tf

2023-05-29 17:05:39.855808: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-29 17:05:39.970909: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-29 17:05:39.970927: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-29 17:05:39.995917: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-29 17:05:40.480262: W tensorflow/stream_executor/platform/de

# Datos

In [2]:
path = os.getcwd()

In [3]:
df_ml_predictores = pd.read_csv('df_ml_predictores.csv', index_col = [0])
df_ml_predictores.set_index(pd.to_datetime(df_ml_predictores.index), inplace = True)

df_ml_target = pd.read_csv('df_ml_target.csv', index_col = [0])
df_ml_target.set_index(pd.to_datetime(df_ml_target.index), inplace = True)

df_2022_predictores = pd.read_csv('df_2022_predictores.csv', index_col = [0])
df_2022_predictores.set_index(pd.to_datetime(df_2022_predictores.index), inplace = True)

df_2022_target = pd.read_csv('df_2022_target.csv', index_col = [0])
df_2022_target.set_index(pd.to_datetime(df_2022_target.index), inplace = True)

In [4]:
df_ml_predictores.index

DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08',
               '2018-01-09', '2018-01-10',
               ...
               '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25',
               '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29',
               '2021-12-30', '2021-12-31'],
              dtype='datetime64[ns]', length=1461, freq=None)

## Entrenamiento y Validación

In [5]:
X_train, X_test, y_train, y_test = train_test_split(df_ml_predictores, df_ml_target, test_size = 0.30, random_state = 42)

# Long Short Term Memory

## 0.- Preprocesamiento LSTM

<img src="time_series.png">

In [6]:
def reshape_for_lstm(data, timesteps, features):
    """
    Reshapes an array into the format required by an LSTM model.

    Args:
        data (numpy.ndarray): Input data array.
        timesteps (int): Number of time steps in the input sequence.
        features (int): Number of features at each time step.

    Returns:
        numpy.ndarray: Reshaped array.
    """
    # Calculate the number of samples
    num_samples = data.shape[0] - timesteps + 1

    # Create an empty array for reshaped data
    reshaped_data = np.zeros((num_samples, timesteps, features))

    # Reshape the data
    for i in range(num_samples):
        reshaped_data[i] = data[i:i + timesteps, :]

    return reshaped_data


In [7]:
n_steps = 3

predictores_lstm = reshape_for_lstm(df_ml_predictores.values,n_steps,df_ml_predictores.shape[1])

target_lstm = df_ml_target.drop(df_ml_target.index[0:n_steps-1])

X_lstm_2022 = reshape_for_lstm(df_2022_predictores.values, n_steps,df_2022_predictores.shape[1])

In [8]:
X_train_lstm, X_test_lstm, Y_train_lstm, Y_test_lstm = train_test_split(predictores_lstm, target_lstm.values, test_size = 0.30, random_state = 42)

## 1.- Arquitectura

In [9]:
model_LSTM = tf.keras.models.Sequential()

model_LSTM.add(tf.keras.layers.LSTM(50,input_shape=(X_train_lstm.shape[1],X_train_lstm.shape[2]), activation = tf.keras.activations.relu))

model_LSTM.add(tf.keras.layers.Dense(1, activation = tf.keras.activations.linear))

model_LSTM.compile(optimizer = 'adam',
                   loss = 'mean_squared_error',
                   metrics = ['mean_absolute_error','mean_squared_error','mean_absolute_percentage_error'])

model_LSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50)                13600     
                                                                 
 dense (Dense)               (None, 1)                 51        
                                                                 
Total params: 13,651
Trainable params: 13,651
Non-trainable params: 0
_________________________________________________________________


2023-05-29 17:05:40.993575: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-29 17:05:40.994022: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-29 17:05:40.994064: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-05-29 17:05:40.994098: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-05-29 17:05:40.994132: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

## 2.- Entrenamiento

In [10]:
lstm_fit = model_LSTM.fit(X_train_lstm, Y_train_lstm, epochs = 250, batch_size = 100)

Epoch 1/250
11/11 [==============================] - 1s 4ms/step - loss: 5525.9526 - mean_absolute_error: 45.8972 - mean_squared_error: 5525.9526 - mean_absolute_percentage_error: 119.0465
Epoch 2/250
11/11 [==============================] - 0s 4ms/step - loss: 3474.8840 - mean_absolute_error: 41.5633 - mean_squared_error: 3474.8840 - mean_absolute_percentage_error: 245.4938
Epoch 3/250
11/11 [==============================] - 0s 4ms/step - loss: 3336.3672 - mean_absolute_error: 41.5748 - mean_squared_error: 3336.3672 - mean_absolute_percentage_error: 250.0446
Epoch 4/250
11/11 [==============================] - 0s 4ms/step - loss: 3044.1147 - mean_absolute_error: 33.5038 - mean_squared_error: 3044.1150 - mean_absolute_percentage_error: 160.9980
Epoch 5/250
11/11 [==============================] - 0s 3ms/step - loss: 2740.6125 - mean_absolute_error: 31.0878 - mean_squared_error: 2740.6125 - mean_absolute_percentage_error: 151.0484
Epoch 6/250
11/11 [==============================] - 0s

In [11]:
fig = plt.figure(figsize = (7*(1+np.sqrt(5))/2,7))
plt.plot(lstm_fit.history['mean_absolute_percentage_error'])
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Percentage Error, %')
plt.title('Evolución del MAPE en el set de entrenamiento')
plt.yscale('log')
#plt.ylim([10**-1,10**3])
#plt.show()

plt.savefig('MAPE_entrenamiento_lstm.png')

## 3.- Evaluación

In [12]:
loss, mae, mse, mape = model_LSTM.evaluate(X_test_lstm,Y_test_lstm)

14/14 [==============================] - 0s 2ms/step - loss: 1009.0453 - mean_absolute_error: 15.6696 - mean_squared_error: 1009.0453 - mean_absolute_percentage_error: 48.6229


## 4.- Predicción

In [13]:
pred_2022_LSTM = model_LSTM.predict(X_lstm_2022)

8/8 [==============================] - 0s 1ms/step


# Comparación de modelos

In [14]:
font = {'family': 'serif',
        'color':  'blue',
        'weight': 'normal',
        'size': 16,
        'alpha': 0.3
        }

fig = plt.figure(figsize = (7*(1+np.sqrt(5))/2,7))

plt.axhline(y = 0, color = 'k', linestyle = '-',linewidth = 1)
plt.axhline(y = 50, color = 'b', linestyle = '-', alpha = 0.3)
plt.axhline(y = 80, color = 'b', linestyle = '-', alpha = 0.3)
plt.axhline(y = 110, color = 'b', linestyle = '-', alpha = 0.3)
plt.axhline(y = 170, color = 'b', linestyle = '-', alpha = 0.3)

plt.plot(df_2022_target, '-k', label = r'Medición')
plt.plot(df_2022_target.index[(n_steps-1)::],pred_2022_LSTM, '-.r', label = r'LSTM')

plt.legend()

plt.text(df_2022_target.index[0], 50, r'Regular', font)
plt.text(df_2022_target.index[0], 80, r'Alerta', font)
plt.text(df_2022_target.index[0], 110, r'Pre-emergencia', font)
plt.text(df_2022_target.index[0], 170, r'Emergencia', font)

plt.title(r'Comparación con datos reales')
plt.xlabel(r'Fechas')
plt.ylabel(r'Concentración de PM2.5, $\mu g/m³$')

#plt.show()

plt.savefig('Comparacion_datos_lstm.png')

In [15]:
print('MAPE del set de entrenamiento: ', np.round(lstm_fit.history['mean_absolute_percentage_error'][-1],3))
print('MAPE del set de test: ', np.round(mape,3))
print('La correlación de la serie de datos con la predicción es: ', np.round(np.corrcoef(np.squeeze(df_2022_target[n_steps-1::].values), np.squeeze(pred_2022_LSTM))[0,1],5))

MAPE del set de entrenamiento:  57.604
MAPE del set de test:  48.623
La correlación de la serie de datos con la predicción es:  0.85345
